## Guide on fine-tuning a Phi-3-mini model for Python code generation utilizing QLoRA via Hugging Face Hub

## Installing and loading the libraries

In [ ]:
# 'pip' is a package installer for Python. It is used to install and manage software packages/libraries written in Python.

# '!pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl flash_attn' is a command that installs or upgrades several Python packages.

# '-qqq' is an option that makes the output of the command less verbose.

# '--upgrade' is an option that upgrades all specified packages to the newest available version.

# 'bitsandbytes' is a package that provides efficient implementations of quantized neural networks.

# 'transformers' is a package that provides state-of-the-art machine learning models like BERT, GPT-2, and RoBERTa.

# 'peft' is a package that provides a Python interface to the Performance API of the Linux kernel.

# 'accelerate' is a package that provides a simple API for distributed and mixed-precision training.

# 'datasets' is a package that provides a simple API for downloading and preparing datasets.

# 'trl' is a package that provides a simple API for reinforcement learning.

# 'flash_attn' is a package that provides a simple API for attention mechanisms in neural networks.

# 'torch' is a package that provides an open-source machine learning library used for building deep learning models.
!pip install -qqq --upgrade bitsandbytes transformers peft accelerate datasets trl torch flash_attn

In [ ]:

# 'pip' is a package installer for Python. It is used to install and manage software packages/libraries written in Python.

# '!pip install huggingface_hub' is a command that installs the 'huggingface_hub' package.
# 'huggingface_hub' is a package that provides a simple API for downloading and uploading files to the Hugging Face Hub.

# '!pip install python-dotenv' is a command that installs the 'python-dotenv' package.
# 'python-dotenv' is a package that allows you to specify environment variables in a .env file, which can be read and used in your Python project.
!pip install huggingface_hub
!pip install python-dotenv

In [ ]:
# 'pip' is a package installer for Python. It is used to install and manage software packages/libraries written in Python.

# '!pip install wandb -qqq' is a command that installs the 'wandb' package.

# 'wandb' (Weights & Biases) is a tool for machine learning experiment tracking, dataset versioning, and model sharing.

# '-qqq' is an option that makes the output of the command less verbose.
!pip install wandb -qqq

In [ ]:
# 'pip' is a package installer for Python. It is used to install and manage software packages/libraries written in Python.

# '!pip install absl-py nltk rouge_score' is a command that installs three Python packages: 'absl-py', 'nltk', and 'rouge_score'.

# 'absl-py' (Abseil Python Common Libraries) is a collection of Python library code that is used within Google to allow Python applications to implement command-line flags and other common utilities.

# 'nltk' (Natural Language Toolkit) is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources.

# 'rouge_score' is a Python package for calculating the ROUGE (Recall-Oriented Understudy for Gisting Evaluation) score, which is a set of metrics used to evaluate automatic summarization and machine translation.
!pip install absl-py nltk rouge_score

In [ ]:
# 'pip' is a package installer for Python. It is used to install and manage software packages/libraries written in Python.

# '!pip list' is a command that lists all installed Python packages.

# '|' is a pipe. It takes the output of the command on its left and uses it as the input to the command on its right.

# 'grep' is a command-line utility for searching plain-text data sets for lines that match a regular expression. In this case, it's used to filter the output of 'pip list'.

# 'transformers.' is the regular expression that 'grep' is searching for. It will match any line that contains 'transformers.'.

# So, '!pip list | grep transformers.' is a command that lists all installed Python packages and filters the list for packages that contain 'transformers.' in their name.
!pip list | grep transformers.

transformers                     4.44.2


## Importing the libraries

In [ ]:
# This code block imports several modules and functions that are needed for the script.

# 'randrange' is a function from the 'random' module that generates a random number within the specified range.

# 'torch' is a library for scientific computing that provides a wide range of functionalities for dealing with tensors.

# 'load_dataset' is a function from the 'datasets' library that loads a dataset from the Hugging Face datasets hub.

# 'LoraConfig', 'prepare_model_for_kbit_training', and 'PeftModel' are classes and functions from the 'peft' library that are used for configuring and preparing the model for training.

# 'AutoModelForCausalLM', 'AutoTokenizer', 'BitsAndBytesConfig', 'TrainingArguments', 'set_seed', and 'pipeline' are classes and functions from the 'transformers' library that are used for creating and training the model.

# 'SFTTrainer' is a class from the 'trl' library that is used for training the model.
from random import randrange

import torch
from datasets import load_dataset

from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed,
    pipeline
)
from trl import SFTTrainer

## Setting Global Parameters

In [ ]:
from google.colab import userdata
import os

os.environ["HF_HUB_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [ ]:
# 'model_id' and 'model_name' are the identifiers for the pre-trained model from Hugging Face hub that you want to fine-tune.
model_id = "microsoft/Phi-3-mini-4k-instruct"
model_name = "microsoft/Phi-3-mini-4k-instruct"

# 'dataset_name' is the identifier for the dataset that you want to use for fine-tuning.
dataset_name = "ostapbodnar/ua-gec-pos-ner-artificial"

# 'dataset_split' is the split of the dataset that you want to use for fine-tuning. In this case, it is set to 'train', which means that the training split of the dataset will be used.
dataset_split= "train"

# 'new_model' is the name that you want to give to the fine-tuned model.
new_model = "Phi3-mini-4k-instruct-UA"

# 'hf_model_repo' is the identifier for the Hugging Face repository where you want to save the fine-tuned model.
hf_model_repo="ostapbodnar/"+new_model

# Load Model on GPU

# 'device_map' is a dictionary that maps devices to model parts. In this case, it is set to {"": 0}, which means that the entire model will be loaded on GPU 0.
device_map = {"": 0}

# Bits and Bytes configuration for the model

# 'use_4bit' is a boolean that controls whether 4-bit precision should be used for loading the base model.
use_4bit = True

# 'bnb_4bit_compute_dtype' is the data type that should be used for computations with the 4-bit base model. In this case, it is set to 'bfloat16'.
bnb_4bit_compute_dtype = "bfloat16"

# 'bnb_4bit_quant_type' is the type of quantization that should be used for the 4-bit base model. In this case, it is set to 'nf4'.
bnb_4bit_quant_type = "nf4"

# 'use_double_quant' is a boolean that controls whether nested quantization should be used for the 4-bit base model.
use_double_quant = True

# LoRA configuration for the model

# 'lora_r' is the dimension of the LoRA attention.
lora_r = 16

# 'lora_alpha' is the alpha parameter for LoRA scaling.
lora_alpha = 16

# 'lora_dropout' is the dropout probability for LoRA layers.
lora_dropout = 0.05

# 'target_modules' is a list of the modules that should be targeted by LoRA.
target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]

# 'set_seed(1234)' sets the random seed for reproducibility.
set_seed(1234)


## Connect to Huggingface Hub

In [ ]:
# 'huggingface_hub' is a library that provides tools for working with the Hugging Face Model Hub.

# 'login' is a function from the 'huggingface_hub' library that logs you into the Hugging Face Model Hub.

# 'dotenv' is a library that allows you to load environment variables from a .env file.

# 'load_dotenv' is a function from the 'dotenv' library that loads environment variables from a .env file.

# 'os' is a standard Python library that provides functions for interacting with the operating system.

# 'os.getenv' is a function from the 'os' library that gets the value of an environment variable.

# 'from huggingface_hub import login' imports the 'login' function from the 'huggingface_hub' library.

# 'from dotenv import load_dotenv' imports the 'load_dotenv' function from the 'dotenv' library.

# 'import os' imports the 'os' library.

# 'load_dotenv()' calls the 'load_dotenv' function, which loads environment variables from a .env file.

# 'login(token=os.getenv("HF_HUB_TOKEN"))' calls the 'login' function with the value of the 'HF_HUB_TOKEN' environment variable as the 'token' parameter. This logs you into the Hugging Face Model Hub using the token stored in the 'HF_HUB_TOKEN' environment variable.
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()
login(token=os.getenv("HF_HUB_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load the dataset with the instruction set

In [ ]:
from datasets import load_dataset, Features, Value

# Define all columns as strings
features = Features({
    'input': Value('string'),
    'output': Value('string'),
    'instruct': Value('string'),
    'dataset_type': Value('string'),
    'dataloader_name': Value('string')
})

In [ ]:
# 'load_dataset' is a function from the 'datasets' library that loads a dataset from the Hugging Face Dataset Hub.

# 'dataset_name' is the identifier for the dataset that you want to load.

# 'split' is the split of the dataset that you want to load. In this case, it is set to the value of 'dataset_split'.

# 'dataset = load_dataset(dataset_name, split=dataset_split)' loads the specified split of the specified dataset from the Hugging Face Dataset Hub and assigns it to the variable 'dataset'.

# 'len(dataset)' returns the size of the dataset.

# 'print(f"dataset size: {len(dataset)}")' prints the size of the dataset.

# 'randrange' is a function from the 'random' library that returns a random integer from the specified range.

# 'len(dataset)' is the range from which 'randrange' should choose a random integer. In this case, it is set to the size of the dataset, which means that 'randrange' will choose a random index from the dataset.

# 'dataset[randrange(len(dataset))]' returns a random example from the dataset.

# 'print(dataset[randrange(len(dataset))])' prints a random example from the dataset.
dataset = load_dataset(dataset_name, data_dir="small", split=dataset_split, features=features)
dataset = dataset.train_test_split(test_size=0.30, seed=1234)['test']
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

dataset size: 28743
{'input': 'Самые распространенные критерии оценки оценки решения кейса: соответствие решения сформулированным вопросам (аяекватность); оригинальность подхода (новаторство, креативность); Таблица 3 Примерная структура итоговой оценки Формы работы студента Доля ыих в общей оценке, % Текущие оценки на практических заняиях Семинар Контрольная работа работа Индивидуальная работа Групповой проект Экзамен Итого: применимость решения на пркатике; Глубина проработанности проблемы (обоснованность и и комплексность решения, наличие альтерантивных ввариантов, прогнозирование сложностей); возможность дальнейшего применения. Для; того читобы в итоговой оценке отразилась значимость каждого Критерия, Им придаются определенные значения, сумма которых должна быть равна единице (таблица 4).', 'output': '<p t="X">Самые</p> <p t="X">распространенные</p> <p t="X">критерии</p> <p t="X">оценки</p><g ed="оценки" et="F/Repetition">оценки оценки</g><p t="X">оценки</p> <p t="X">решения</p> <p 

In [ ]:
# 'dataset' is a variable that contains the dataset loaded from the Hugging Face Dataset Hub.

# 'dataset' when used alone like this in a Jupyter notebook cell, it will display the structure of the dataset. This includes information such as the number of examples in the dataset, the names and types of the fields in the dataset, and the shapes of the fields.

# This line of code is used to check the structure of the dataset to ensure that it is in the expected format before proceeding with further data processing or model training.
dataset

Dataset({
    features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
    num_rows: 28743
})

In [ ]:
# 'randrange' is a function from the 'random' library that returns a random integer from the specified range.

# 'len(dataset)' is the range from which 'randrange' should choose a random integer. In this case, it is set to the size of the dataset, which means that 'randrange' will choose a random index from the dataset.

# 'dataset[randrange(len(dataset))]' returns a random example from the dataset.

# 'print(dataset[randrange(len(dataset))])' prints a random example from the dataset. This is useful for getting a sense of what the data in the dataset looks like.
print(dataset[randrange(len(dataset))])

{'input': 'В психологической науке и практике достаточно широко рассматриваются вопросы профессионального отбора отбора в областях, связанных с действием стрессогенных факторов, таких как риск для здоровья или жизни, работа в сложных, необычных условиях, требующая высокой концентрац?и вимания, памяти. Психологическая диагностика и психологическое сопровождение в этих областях профессиональной деятельности позволяют снизить уровень неблагоприятных последствий для психического и физического здоровья; людей, количество допускаемых ошибок, которые являются потенциально омасными длф окружающих и могу привести к маериальным потерям- В таоих стрессогенных областях профессиональной деятельности психологическая диагностика является необходимой процедурой при приеме на Работу сотрудников, а такге при отборе Кандидатов для профессионального обученияж', 'output': '<p t="ADP">В</p> <p t="ADJ">психологической</p> <p t="X">науке</p> <p t="X">и</p> <p t="X">практике</p> <p t="X">достаточно</p> <p t="X

## Load the tokenizer to prepare the dataset

In [ ]:
# 'tokenizer_id' is the identifier for the tokenizer that you want to load. In this case, it is set to the value of 'model_id', which means that the tokenizer associated with the pre-trained model will be loaded.

# 'AutoTokenizer' is a class from the 'transformers' library that provides a generic tokenizer class from which all other tokenizer classes inherit.

# 'from_pretrained' is a method of the 'AutoTokenizer' class that loads a tokenizer from the Hugging Face Model Hub.

# 'tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)' loads the tokenizer associated with 'tokenizer_id' from the Hugging Face Model Hub and assigns it to the variable 'tokenizer'.

# 'tokenizer.padding_side' is a property of the 'tokenizer' object that determines on which side of the input sequences padding should be added. It can be set to either 'left' or 'right'.

# 'tokenizer.padding_side = 'right'' sets 'tokenizer.padding_side' to 'right', which means that padding will be added to the right side of the input sequences. This is done to prevent warnings that can occur when 'tokenizer.padding_side' is set to 'left'.
tokenizer_id = model_id
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
tokenizer.padding_side = 'right' # to prevent warnings

Function to generate the suitable format for our model.

In [ ]:
# 'create_message_column' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'messages' and the value is a list of dictionaries, each representing a message.

# 'row' is the input to the 'create_message_column' function. It is expected to be a dictionary with keys 'instruction', 'input', and 'output'.

# 'messages' is a list that will contain the messages.

# 'user' is a dictionary that represents a user message. The 'content' key contains the instruction and input from the row, and the 'role' key is set to 'user'.

# 'messages.append(user)' adds the user message to the 'messages' list.

# 'assistant' is a dictionary that represents an assistant message. The 'content' key contains the output from the row, and the 'role' key is set to 'assistant'.

# 'messages.append(assistant)' adds the assistant message to the 'messages' list.

# 'return {"messages": messages}' returns a dictionary with a single key-value pair. The key is 'messages' and the value is the 'messages' list.

# 'format_dataset_chatml' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row.

# 'row' is the input to the 'format_dataset_chatml' function. It is expected to be a dictionary with a key 'messages'.

# 'return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}' returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row. The 'add_generation_prompt' parameter is set to False, which means that no generation prompt will be added to the end of the text. The 'tokenize' parameter is set to False, which means that the text will not be tokenized.
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruct']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

Implement the ChatML format on our dataset.

In [ ]:
# 'dataset' is a variable that contains the dataset loaded from the Hugging Face Dataset Hub.

# 'map' is a method of the 'Dataset' class that applies a function to each example in the dataset.

# 'create_message_column' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'messages' and the value is a list of dictionaries, each representing a message.

# 'dataset_chatml = dataset.map(create_message_column)' applies the 'create_message_column' function to each example in 'dataset' and assigns the result to 'dataset_chatml'. This transforms the dataset into a format where each example is a list of messages.

# 'format_dataset_chatml' is a function that takes a row from a dataset and returns a dictionary with a single key-value pair. The key is 'text' and the value is the result of applying the chat template to the messages in the row.

# 'dataset_chatml = dataset_chatml.map(format_dataset_chatml)' applies the 'format_dataset_chatml' function to each example in 'dataset_chatml'. This transforms the dataset into a format where each example is a single string of text that represents a conversation.
dataset_chatml = dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

Map:   0%|          | 0/28743 [00:00<?, ? examples/s]

Map:   0%|          | 0/28743 [00:00<?, ? examples/s]

In [ ]:
dataset_chatml.cleanup_cache_files()

7

In [ ]:
dataset.cleanup_cache_files()

1

In [ ]:
# 'dataset_chatml' is a variable that contains the dataset that has been transformed into a format where each example is a single string of text that represents a conversation.

# 'dataset_chatml[0]' returns the first example in 'dataset_chatml'.

# This line of code is used to check the first example in the transformed dataset to ensure that the transformation was performed correctly.
dataset_chatml[0]

{'input': 'Проте таке повідомлення, як і документ про оплату послуг пересилання поштового відправлення, ТОВ "ФК "Довіра Та гарантія"й разом Із іншими документами для проведення державної реєстрації права не пойало, тому вимоги Порядку державної реєстрації речових прав иа нерухоме майно та їх обтяжень, затвердженого постановою Кабінету Міністрів України від 25 грудн 2015 року №1127 (далі - Порядок), щодо надання документа, що підтверджує завершення 30 - денного строку з моменту отримання іпотекодавцем та боржником письмової вимоги іпотекодержателя, не виконано. Відмовляючи в задоволенні позовних вимог про визнання за позивачем прав власності на квартиру, шуд виходив з того, що при скасуванні рішення державного реєстратора КП "Центр розвитку та інвестицій Васильківського району" Пархомчука- Ю. А. про державну реєстрацію прав та їх обтяжень (з відкриттям розділу) та зпису про державну реєстрацію права власності на квартиру повторного взнання за позивачем Прав власності на квартиру законом

In [ ]:
# 'dataset_chatml' is a variable that contains the dataset that has been transformed into a format where each example is a single string of text that represents a conversation.

# 'train_test_split' is a method of the 'Dataset' class that splits the dataset into a training set and a test set.

# 'test_size=0.05' is a parameter of the 'train_test_split' method that specifies the proportion of the dataset to include in the test set. In this case, it is set to 0.05, which means that 5% of the dataset will be included in the test set.

# 'seed=1234' is a parameter of the 'train_test_split' method that specifies the seed for the random number generator. This is used to ensure that the split is reproducible.

# 'dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)' splits 'dataset_chatml' into a training set and a test set and assigns the result to 'dataset_chatml'. The result is a dictionary with two key-value pairs. The keys are 'train' and 'test', and the values are the training set and the test set, respectively.

# 'dataset_chatml' when used alone like this in a Jupyter notebook cell, it will display the structure of the training set and the test set. This includes information such as the number of examples in each set, the names and types of the fields in the sets, and the shapes of the fields.

# This line of code is used to check the structure of the training set and the test set to ensure that the split was performed correctly.
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05, seed=1234)
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 27305
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 1438
    })
})

## Instruction fine-tune a Phi-3-mini model using QLORA and trl

Initially, we attempt to recognize our GPU.

In [ ]:
# 'torch' is a library for scientific computing that provides a wide range of functionalities for dealing with tensors, which are multi-dimensional arrays.

# 'torch.cuda.is_bf16_supported()' is a function that checks if BF16 is supported on the current GPU. BF16 is a data type that uses 16 bits, like float16, but allocates more bits to the exponent, which can result in higher precision.

# 'compute_dtype' is a variable that will hold the data type to be used for computations.

# 'attn_implementation' is a variable that will hold the type of attention implementation to be used.

# 'if torch.cuda.is_bf16_supported():' checks if BF16 is supported on the current GPU. If it is, the following block of code is executed.

# 'compute_dtype = torch.bfloat16' sets 'compute_dtype' to 'torch.bfloat16', which is the BF16 data type in PyTorch.

# 'attn_implementation = 'flash_attention_2'' sets 'attn_implementation' to 'flash_attention_2', which is a type of attention implementation.

# 'else:' specifies that the following block of code should be executed if BF16 is not supported on the current GPU.

# 'compute_dtype = torch.float16' sets 'compute_dtype' to 'torch.float16', which is the float16 data type in PyTorch.

# 'attn_implementation = 'sdpa'' sets 'attn_implementation' to 'sdpa', which is a type of attention implementation.

# 'print(attn_implementation)' prints the value of 'attn_implementation', which is the type of attention implementation to be used.

# 'print(compute_dtype)' prints the value of 'compute_dtype', which is the data type to be used for computations.
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

print(attn_implementation)
print(compute_dtype)

flash_attention_2
torch.bfloat16


## Load the tokenizer and model to finetune

In [ ]:
# 'AutoTokenizer' is a class from the Hugging Face Transformers library that provides a tokenizer for a given pre-trained model.

# 'from_pretrained' is a method of the 'AutoTokenizer' class that loads a tokenizer from a pre-trained model.

# 'model_name' is a variable that contains the name of the pre-trained model.

# 'trust_remote_code=True' is a parameter that allows the execution of remote code when loading the tokenizer.

# 'add_eos_token=True' is a parameter that adds an end-of-sentence token to the tokenizer.

# 'use_fast=True' is a parameter that uses the fast version of the tokenizer, if available.

# 'tokenizer.pad_token = tokenizer.unk_token' sets the padding token of the tokenizer to be the same as the unknown token.

# 'tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)' sets the ID of the padding token to be the same as the ID of the padding token.

# 'tokenizer.padding_side = 'left'' sets the side where padding will be added to be the left side.

# 'BitsAndBytesConfig' is a class that provides a configuration for quantization.

# 'bnb_config' is a variable that holds the configuration for quantization.

# 'AutoModelForCausalLM' is a class from the Hugging Face Transformers library that provides a model for causal language modeling.

# 'from_pretrained' is a method of the 'AutoModelForCausalLM' class that loads a model from a pre-trained model.

# 'torch_dtype=compute_dtype' is a parameter that sets the data type of the model to be the same as 'compute_dtype'.

# 'quantization_config=bnb_config' is a parameter that sets the configuration for quantization to be 'bnb_config'.

# 'device_map=device_map' is a parameter that sets the device map of the model to be 'device_map'.

# 'attn_implementation=attn_implementation' is a parameter that sets the type of attention implementation to be 'attn_implementation'.

# 'prepare_model_for_kbit_training' is a function that prepares a model for k-bit training.

# 'model = prepare_model_for_kbit_training(model)' prepares 'model' for k-bit training and assigns the result back to 'model'.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'left'

bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_double_quant,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name, torch_dtype=compute_dtype, trust_remote_code=True, quantization_config=bnb_config, device_map=device_map,
          attn_implementation=attn_implementation
)

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Set up the QLoRA parameters.

In [ ]:
# This code block is used to define the training arguments for the model.

# 'TrainingArguments' is a class that holds the arguments for training a model.
# 'output_dir' is the directory where the model and its checkpoints will be saved.
# 'evaluation_strategy' is set to "steps", meaning that evaluation will be performed after a certain number of training steps.
# 'do_eval' is set to True, meaning that evaluation will be performed.
# 'optim' is set to "adamw_torch", meaning that the AdamW optimizer from PyTorch will be used.
# 'per_device_train_batch_size' and 'per_device_eval_batch_size' are set to 8, meaning that the batch size for training and evaluation will be 8 per device.
# 'gradient_accumulation_steps' is set to 4, meaning that gradients will be accumulated over 4 steps before performing a backward/update pass.
# 'log_level' is set to "debug", meaning that all log messages will be printed.
# 'save_strategy' is set to "epoch", meaning that the model will be saved after each epoch.
# 'logging_steps' is set to 100, meaning that log messages will be printed every 100 steps.
# 'learning_rate' is set to 1e-4, which is the learning rate for the optimizer.
# 'fp16' is set to the opposite of whether bfloat16 is supported on the current CUDA device.
# 'bf16' is set to whether bfloat16 is supported on the current CUDA device.
# 'eval_steps' is set to 100, meaning that evaluation will be performed every 100 steps.
# 'num_train_epochs' is set to 3, meaning that the model will be trained for 3 epochs.
# 'warmup_ratio' is set to 0.1, meaning that 10% of the total training steps will be used for the warmup phase.
# 'lr_scheduler_type' is set to "linear", meaning that a linear learning rate scheduler will be used.
# 'report_to' is set to "wandb", meaning that training and evaluation metrics will be reported to Weights & Biases.
# 'seed' is set to 42, which is the seed for the random number generator.

# LoraConfig object is created with the following parameters:
# 'r' (rank of the low-rank approximation) is set to 16,
# 'lora_alpha' (scaling factor) is set to 16,
# 'lora_dropout' dropout probability for Lora layers is set to 0.05,
# 'task_type' (set to TaskType.CAUSAL_LM indicating the task type),
# 'target_modules' (the modules to which LoRA is applied) choosing linear layers except the output layer..


args = TrainingArguments(
        output_dir="./phi-3-mini-QLoRA",
        evaluation_strategy="steps",
        hub_model_id="ostapbodnar/Phi3-mini-4k-instruct-UA-adapter"
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=4,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=1000,
        learning_rate=1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        eval_steps=1000,
        num_train_epochs=4,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        report_to="wandb",
        seed=42,
)

peft_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        task_type=TaskType.CAUSAL_LM,
        target_modules=target_modules,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


The SFTTrainer offers a built-in integration with peft, simplifying the process of effectively fine-tuning LLMs. All we need to do is establish our LoRAConfig and supply it to the trainer. However, before initiating our training, it's necessary to specify the hyperparameters (TrainingArguments) we plan to utilize.

## Establish a connection with wandb and enlist the project and experiment.

In [ ]:
# 'wandb' is a library for machine learning experiment tracking, dataset versioning, and model management.

# 'import wandb' is a line of code that imports the 'wandb' library.

# 'wandb.login()' is a function that logs you into your Weights & Biases account. If you're not logged in, it will prompt you to enter your API key.

# This block of code is used to initialize Weights & Biases for experiment tracking.
import wandb
wandb.login(key=os.environ["WANDB_API_KEY"])

wandb: Currently logged in as: ostapbodnar (ostap-bodnar). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# 'os' is a standard Python library that provides functions for interacting with the operating system.

# 'import os' is a line of code that imports the 'os' library.

# 'os.environ' is a mapping object representing the string environment. It's used to get and set the environment variables in the operating system.

# 'os.environ["_PROJECT"]="ProjectName"' is a line of code that sets the environment variable '_PROJECT' to 'ProjectName'. This can be useful for setting configuration variables that your program needs to run.

# This block of code is used to set the '_PROJECT' environment variable to 'ProjectName'.
import os

os.environ["PROJECT"]="phi3-mini-4k-qlora-fn"

In [ ]:
# 'project_name' is a variable that holds the name of the project.

# 'wandb.init' is a function from the 'wandb' library that initializes a new Weights & Biases run. A run represents a single execution of your script.

# 'project=project_name' is a parameter that sets the project name for the Weights & Biases run to be 'project_name'.

# 'name = "ProjectName"' is a parameter that sets the name of the Weights & Biases run to be 'ProjectName'. This name will be displayed in the Weights & Biases dashboard.

# This block of code is used to initialize a new Weights & Biases run with the specified project name and run name.
project_name = os.environ["PROJECT"]

wandb.init(project=project_name, name = project_name)

We now possess all the necessary components to construct our SFTTrainer and commence the model training.

In [ ]:
# 'SFTTrainer' is a class that provides a trainer for fine-tuning a model.

# 'trainer' is a variable that holds the trainer.

# 'model=model' is a parameter that sets the model to be trained to be 'model'.

# 'train_dataset=dataset_chatml['train']' is a parameter that sets the training dataset to be 'dataset_chatml['train']'.

# 'eval_dataset=dataset_chatml['test']' is a parameter that sets the evaluation dataset to be 'dataset_chatml['test']'.

# 'peft_config=peft_config' is a parameter that sets the configuration for the Lora layer to be 'peft_config'.

# 'dataset_text_field="text"' is a parameter that sets the field in the dataset that contains the text to be 'text'.

# 'max_seq_length=512' is a parameter that sets the maximum sequence length for the model to be 512.

# 'tokenizer=tokenizer' is a parameter that sets the tokenizer to be 'tokenizer'.

# 'args=args' is a parameter that sets the training arguments to be 'args'.

# This line of code is used to create a trainer for fine-tuning the model with the specified parameters.
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_chatml['train'],
        eval_dataset=dataset_chatml['test'],
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=2048,
        tokenizer=tokenizer,
        args=args,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/27305 [00:00<?, ? examples/s]

Map:   0%|          | 0/1438 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


Initiate the model training process by invoking the train() method on our Trainer instance.

In [ ]:
torch.cuda.empty_cache()

In [29]:
# 'trainer.train()' is a method that starts the training of the model. It uses the training dataset, model, and training arguments that were specified when the trainer was created.

# 'trainer.save_model()' is a method that saves the trained model to the local file system. The model will be saved in the output directory that was specified in the training arguments.

# This block of code is used to train the model and then save the trained model to the local file system.
# train
trainer.train()

# save model in local
trainer.save_model()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 27,305
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 13,652
  Number of trainable parameters = 8,912,896
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pa

Step,Training Loss,Validation Loss
1000,0.844100,0.605920
2000,0.580600,0.560083
3000,0.550900,0.537082
4000,0.529300,0.523148
5000,0.518700,0.512125
6000,0.506600,0.504143
7000,0.501000,0.498816



***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4

***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4

***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4
Saving model checkpoint to ./phi-3-mini-QLoRA/checkpoint-3413
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size"

Step,Training Loss,Validation Loss
1000,0.844100,0.605920
2000,0.580600,0.560083
3000,0.550900,0.537082
4000,0.529300,0.523148
5000,0.518700,0.512125
6000,0.506600,0.504143
7000,0.501000,0.498816
8000,0.490400,0.493766
9000,0.488900,0.490325
10000,0.487100,0.487111



***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4

***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4

***** Running Evaluation *****
  Num examples = 1438
  Batch size = 4
Saving model checkpoint to ./phi-3-mini-QLoRA/checkpoint-10240
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size

In [30]:
# 'hf_adapter_repo' is a variable that holds the repository name for the Hugging Face model adapter.

# 'username/model-adapter-name' is the repository name, where 'username' is the username of the repository owner and 'model-adapter-name' is the name of the model adapter.

# This line of code is used to set the repository name for the Hugging Face model adapter.
hf_adapter_repo="ostapbodnar/Phi3-mini-4k-instruct-UA-adapter"

In [31]:
# 'trainer.push_to_hub(hf_adapter_repo)' is a method that pushes the trained model adapter to the Hugging Face Model Hub.

# 'hf_adapter_repo' is the repository name for the model adapter on the Hugging Face Model Hub.

# This line of code is used to save the trained model adapter to the Hugging Face Model Hub.
# Save the adapter
trainer.push_to_hub(hf_adapter_repo)

Saving model checkpoint to ./phi-3-mini-QLoRA
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ostapbodnar/phi-3-mini-QLoRA/commit/b8b29c5ec487a0c4cd2164eec3a117665a2d010d', commit_message='ostapbodnar/Phi3-mini-4k-instruct-UA-adapter', commit_description='', oid='b8b29c5ec487a0c4cd2164eec3a117665a2d010d', pr_url=None, pr_revision=None, pr_num=None)

## Combine the model and the adapters, then save it.

Note: When operating on a T4 instance, memory cleanup is necessary.

In [32]:
# 'del model' and 'del trainer' are lines of code that delete the 'model' and 'trainer' objects. This frees up the memory that was used by these objects.

# 'import gc' is a line of code that imports the 'gc' module, which provides an interface to the garbage collector.

# 'gc.collect()' is a function that triggers a full garbage collection. It frees up memory by collecting all the objects that are no longer in use.

# This block of code is used to empty the VRAM (Video Random Access Memory) by deleting the 'model' and 'trainer' objects and then triggering a full garbage collection.
# Empty VRAM
del model
del trainer
import gc
gc.collect()
gc.collect()

22079

In [33]:
# 'torch.cuda.empty_cache()' is a function from the PyTorch library that releases all unoccupied cached memory currently held by the caching allocator so that those can be used in other GPU application and visible in nvidia-smi.

# It's a PyTorch specific function to manage GPU memory and it doesn't affect the GPU memory usage by PyTorch tensors.

# This line of code is used to empty the cache memory that's used by PyTorch on the GPU.
torch.cuda.empty_cache() # PyTorch thing

In [34]:
# 'torch.cuda.empty_cache()' is a function from the PyTorch library that releases all unoccupied cached memory currently held by the caching allocator so that those can be used in other GPU application and visible in nvidia-smi.

# It's a PyTorch specific function to manage GPU memory and it doesn't affect the GPU memory usage by PyTorch tensors.

# This line of code is used to empty the cache memory that's used by PyTorch on the GPU.
torch.cuda.empty_cache() # PyTorch thing

Reload the previously trained and saved model, merge it, and then proceed to save the entire model.

In [37]:
# 'hf_adapter_repo' is a variable that holds the repository name for the Hugging Face model adapter.

# 'edumunozsala/phi-3-mini-QLoRA' is the repository name, where 'edumunozsala' is the username of the repository owner and 'phi-3-mini-QLoRA' is the name of the model adapter.

# 'model_name, hf_adapter_repo, compute_dtype' is a line of code that returns the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.

# This block of code is used to set the repository name for the Hugging Face model adapter and then return the values of the 'model_name', 'hf_adapter_repo', and 'compute_dtype' variables.
hf_adapter_repo = "ostapbodnar/Phi3-mini-4k-instruct-UA-adapter"

model_name, hf_adapter_repo, compute_dtype

('microsoft/Phi-3-mini-4k-instruct',
 'ostapbodnar/Phi3-mini-4k-instruct-UA-adapter',
 torch.bfloat16)

In [38]:
# 'peft_model_id' and 'tr_model_id' are variables that hold the identifiers for the PEFT model and the transformer model, respectively.

# 'AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=compute_dtype)' is a function that loads a pre-trained transformer model for causal language modeling. 'tr_model_id' is the identifier for the pre-trained model, 'trust_remote_code=True' allows the execution of code from the model file, and 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors.

# 'PeftModel.from_pretrained(model, peft_model_id)' is a function that loads a pre-trained PEFT model. 'model' is the transformer model and 'peft_model_id' is the identifier for the pre-trained PEFT model.

# 'model.merge_and_unload()' is a method that merges the PEFT model with the transformer model and then unloads the PEFT model.

# This block of code is used to load a pre-trained transformer model and a pre-trained PEFT model, merge the two models, and then unload the PEFT model.
peft_model_id = hf_adapter_repo
tr_model_id = model_name

model = AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=compute_dtype)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.merge_and_unload()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_atte

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/5a516f86087853f9d560c95eb9209c1d4ed9ff69/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

In [39]:
# 'tokenizer' is a variable that holds the tokenizer.

# 'AutoTokenizer.from_pretrained(peft_model_id)' is a function from the Hugging Face Transformers library that loads a pre-trained tokenizer. 'peft_model_id' is the identifier for the pre-trained tokenizer.

# This line of code is used to load a pre-trained tokenizer.
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

tokenizer_config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA-adapter/snapshots/b8b29c5ec487a0c4cd2164eec3a117665a2d010d/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA-adapter/snapshots/b8b29c5ec487a0c4cd2164eec3a117665a2d010d/tokenizer.json
loading file added_tokens.json from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA-adapter/snapshots/b8b29c5ec487a0c4cd2164eec3a117665a2d010d/added_tokens.json
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA-adapter/snapshots/b8b29c5ec487a0c4cd2164eec3a117665a2d010d/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA-adapter/snapshots/b8b29c5ec487a0c4cd2164eec3a117665a2d010d/tokenizer_config.json
Speci

In [40]:
# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model.

# This line of code is used to reference the repository name for the Hugging Face model.
hf_model_repo

'ostapbodnar/Phi3-mini-4k-instruct-UA'

In [41]:
# 'merged_model_id' is a variable that holds the identifier for the merged model.

# 'hf_model_repo' is the repository name for the Hugging Face model.

# 'model.push_to_hub(merged_model_id)' is a method that pushes the merged model to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the merged model.

# 'tokenizer.push_to_hub(merged_model_id)' is a method that pushes the tokenizer to the Hugging Face Model Hub. 'merged_model_id' is the identifier for the tokenizer.

# This block of code is used to save the merged model and the tokenizer to the Hugging Face Model Hub.
# SAve the model merged to the Hub
merged_model_id = hf_model_repo
model.push_to_hub(merged_model_id)
tokenizer.push_to_hub(merged_model_id)

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Configuration saved in /tmp/tmpmkeau_kh/config.json
Configuration saved in /tmp/tmpmkeau_kh/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 2 checkpoint shards. You can find where each parameters has been saved in the index located at /tmp/tmpmkeau_kh/model.safetensors.index.json.
Uploading the following files to ostapbodnar/Phi3-mini-4k-instruct-UA: generation_config.json,README.md,model.safetensors.index.json,config.json,model-00002-of-00002.safetensors,model-00001-of-00002.safetensors


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmpm9q6xf55/tokenizer_config.json
Special tokens file saved in /tmp/tmpm9q6xf55/special_tokens_map.json
Uploading the following files to ostapbodnar/Phi3-mini-4k-instruct-UA: tokenizer.json,README.md,tokenizer_config.json,added_tokens.json,special_tokens_map.json,tokenizer.model


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ostapbodnar/Phi3-mini-4k-instruct-UA/commit/6228f6a1e46c26a8d6d6dc328df7d16c90cadc14', commit_message='Upload tokenizer', commit_description='', oid='6228f6a1e46c26a8d6d6dc328df7d16c90cadc14', pr_url=None, pr_revision=None, pr_num=None)

## Model Inference and evaluation

In the end, we download the model that we created from the hub and conduct tests to ensure its proper functioning!

In [42]:
# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model.

# This line of code is used to reference the repository name for the Hugging Face model.
hf_model_repo

'ostapbodnar/Phi3-mini-4k-instruct-UA'

In [43]:
# 'hf_model_repo' is a variable that holds the repository name for the Hugging Face model.

# 'username/modelname' is the repository name, where 'username' is the username of the repository owner and 'modelname' is the name of the model.

# This line of code is used to set the repository name for the Hugging Face model.
hf_model_repo='ostapbodnar/Phi3-mini-4k-instruct-UA'

Retrieve the model and tokenizer from the Hub.

In [44]:
# 'device_map' is a variable that holds the mapping of the devices that are used for computation.

# 'compute_dtype' is a variable that holds the data type that is used for computation.

# This line of code is used to return the values of the 'device_map' and 'compute_dtype' variables.
device_map, compute_dtype

({'': 0}, torch.bfloat16)

In [45]:
# This block of code is used to import the necessary libraries, set the seed for reproducibility, and load a pre-trained tokenizer and model.

# 'import torch' is a line of code that imports the PyTorch library, which is a popular open-source machine learning library.

# 'from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed' is a line of code that imports the 'AutoTokenizer', 'AutoModelForCausalLM', and 'set_seed' functions from the Hugging Face Transformers library.

# 'set_seed(1234)' is a line of code that sets the seed for the random number generator to '1234'. This is done to ensure that the results are reproducible.

# 'tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)' is a line of code that loads a pre-trained tokenizer from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model and 'trust_remote_code=True' allows the execution of code from the model file.

# 'model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map)' is a line of code that loads a pre-trained model for causal language modeling from the Hugging Face Model Hub. 'hf_model_repo' is the repository name for the model, 'trust_remote_code=True' allows the execution of code from the model file, 'torch_dtype=compute_dtype' sets the data type for the PyTorch tensors, and 'device_map=device_map' sets the device mapping.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

set_seed(1234)  # For reproducibility

tokenizer = AutoTokenizer.from_pretrained(hf_model_repo,trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, trust_remote_code=True, torch_dtype=compute_dtype, device_map=device_map) # compute "auto" dev_map "cuda"

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

...

Instantiating Phi3ForCausalLM model under default dtype torch.bfloat16.
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "pad_token_id": 32000
}



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at ostapbodnar/Phi3-mini-4k-instruct-UA.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.


generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--ostapbodnar--Phi3-mini-4k-instruct-UA/snapshots/6228f6a1e46c26a8d6d6dc328df7d16c90cadc14/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}



We arrange the dataset in the same manner as before.

In [46]:
# This block of code is used to prepare the dataset for training.

# 'dataset_chatml = dataset.map(create_message_column)' is a line of code that applies the 'create_message_column' function to each element in the 'dataset'. The result is stored in 'dataset_chatml'.

# 'dataset_chatml = dataset_chatml.map(format_dataset_chatml)' is a line of code that applies the 'format_dataset_chatml' function to each element in 'dataset_chatml'.

# 'dataset_chatml = dataset_chatml.train_test_split(test_size=0.05)' is a line of code that splits 'dataset_chatml' into a training set and a test set. The test set size is 5% of the total dataset.

# 'dataset_chatml' is a line of code that returns the 'dataset_chatml' variable.

# This block of code is used to create a message column in the dataset, format the dataset, and then split the dataset into a training set and a test set.
## prepare the dataset
dataset_chatml = dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)
dataset_chatml = dataset_chatml.train_test_split(test_size=0.05)
dataset_chatml

Map:   0%|          | 0/28743 [00:00<?, ? examples/s]

Map:   0%|          | 0/28743 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 27305
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 1438
    })
})

Establish a text generation pipeline to execute the inference.

In [47]:
# 'pipe' is a variable that holds the pipeline for text generation.

# 'pipeline("text-generation", model=model, tokenizer=tokenizer)' is a function from the Hugging Face Transformers library that creates a pipeline for text generation. 'model' is the pre-trained model and 'tokenizer' is the pre-trained tokenizer.

# This line of code is used to create a pipeline for text generation using the pre-trained model and tokenizer.
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [68]:
# 'pipe' is an instance of the 'pipeline' class from the 'transformers' library. It is used to generate responses to prompts.

# 'pipe.tokenizer' is the tokenizer of the 'pipe' pipeline. It is used to convert text into a format that the model can understand.

# 'apply_chat_template' is a function of the tokenizer that formats a chat message in a way that the model can understand.

# '[{"role": "user", "content": dataset_chatml['test'][0]['messages'][0]['content']}]' is a list of dictionaries that represents a chat message. Each dictionary contains a 'role' field that indicates the role of the sender (in this case, 'user') and a 'content' field that contains the text of the message.

# 'tokenize=False' is a parameter that controls whether the 'apply_chat_template' function should tokenize the input. In this case, it is set to 'False', which means that the function will not tokenize the input.

# 'add_generation_prompt=True' is a parameter that controls whether the 'apply_chat_template' function should add a generation prompt to the input. In this case, it is set to 'True', which means that the function will add a generation prompt.

# This line of code tests the 'apply_chat_template' function by applying it to the first message in the test set of the 'dataset_chatml' dataset.
pipe.tokenizer.apply_chat_template([{"role": "user", "content": dataset_chatml['test'][33]['messages'][0]['content']}], tokenize=False, add_generation_prompt=True)

'<|user|>\nВикористай наведені нижче інструкції для обробки тексту.\r\n\r\n    1. Виправлення граматичних помилок:\r\n       - Формат: `<g ed="edit" et="тип помилки">error</g>`\r\n       - Опис тегів:\r\n         - Tag: категорія помилки, наприклад, G/Case, G/Tense, F/Style тощо.\r\n         - error: текст до виправлення.\r\n         - edit: текст після виправлення.\r\n       - Типи помилок:\r\n         - Орфографічні помилки:\r\n           - Spelling: орфографічні помилки.\r\n           - Punctuation: пунктуаційні помилки.\r\n         - Граматичні помилки:\r\n           - G/Case: неправильне використання відмінка.\r\n           - G/Gender: неправильне використання роду.\r\n           - G/Number: неправильне використання числа.\r\n           - G/Aspect: неправильне використання аспекта дієслова.\r\n           - G/Tense: неправильне використання часу дієслова.\r\n           - G/VerbVoice: неправильне використання форми дієслова.\r\n           - G/PartVoice: неправильне використання форм

In [70]:
# 'test_inference' is a function that generates a response to a given prompt.

# 'prompt' is the input parameter, which is expected to be a string that contains the user's message.

# 'prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)' applies a chat template to the 'prompt' and assigns the result back to 'prompt'. The 'apply_chat_template' function is part of the tokenizer of the 'pipe' pipeline. It formats the 'prompt' in a way that the model can understand.

# 'outputs = pipe(prompt, max_new_tokens=256, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time= 180)' generates a response to the 'prompt' using the 'pipe' pipeline. The parameters control the generation process. For example, 'max_new_tokens=256' limits the length of the generated response to 256 tokens, and 'temperature=0.3' controls the randomness of the generation process (lower values make the output more deterministic).

# 'return outputs[0]['generated_text'][len(prompt):].strip()' returns the generated response. The '[len(prompt):]' part removes the original 'prompt' from the beginning of the 'generated_text', and the 'strip()' part removes leading and trailing whitespace.
def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=2048, do_sample=True, num_beams=1, temperature=0.3, top_k=50, top_p=0.95, max_time= 180)
    return outputs[0]['generated_text'].strip()

In [71]:
dataset_chatml['test'][33]['messages'][0]['content']

'Використай наведені нижче інструкції для обробки тексту.\r\n\r\n    1. Виправлення граматичних помилок:\r\n       - Формат: `<g ed="edit" et="тип помилки">error</g>`\r\n       - Опис тегів:\r\n         - Tag: категорія помилки, наприклад, G/Case, G/Tense, F/Style тощо.\r\n         - error: текст до виправлення.\r\n         - edit: текст після виправлення.\r\n       - Типи помилок:\r\n         - Орфографічні помилки:\r\n           - Spelling: орфографічні помилки.\r\n           - Punctuation: пунктуаційні помилки.\r\n         - Граматичні помилки:\r\n           - G/Case: неправильне використання відмінка.\r\n           - G/Gender: неправильне використання роду.\r\n           - G/Number: неправильне використання числа.\r\n           - G/Aspect: неправильне використання аспекта дієслова.\r\n           - G/Tense: неправильне використання часу дієслова.\r\n           - G/VerbVoice: неправильне використання форми дієслова.\r\n           - G/PartVoice: неправильне використання форми дієприкм

In [72]:
# '%%time' is a magic command in Jupyter Notebook that measures the execution time of the cell it is placed in.

# 'test_inference' is a function that generates a response to a given input.

# 'dataset_chatml' is a dictionary-like object that contains the dataset.

# 'dataset_chatml['test']' retrieves the test set from the 'dataset_chatml' dataset.

# 'dataset_chatml['test'][0]' retrieves the first sample from the test set.

# 'dataset_chatml['test'][0]['messages'][0]['content']' retrieves the 'content' field from the first message in the 'messages' field of the first sample in the test set.

# 'test_inference(dataset_chatml['test'][0]['messages'][0]['content'])' generates a response to the 'content' field of the first message in the 'messages' field of the first sample in the test set.
%%time

test_inference(dataset_chatml['test'][33]['messages'][0]['content'])

CPU times: user 4.23 s, sys: 35 ms, total: 4.27 s
Wall time: 4.26 s


'<|user|>\nВикористай наведені нижче інструкції для обробки тексту.\r\n\r\n    1. Виправлення граматичних помилок:\r\n       - Формат: `<g ed="edit" et="тип помилки">error</g>`\r\n       - Опис тегів:\r\n         - Tag: категорія помилки, наприклад, G/Case, G/Tense, F/Style тощо.\r\n         - error: текст до виправлення.\r\n         - edit: текст після виправлення.\r\n       - Типи помилок:\r\n         - Орфографічні помилки:\r\n           - Spelling: орфографічні помилки.\r\n           - Punctuation: пунктуаційні помилки.\r\n         - Граматичні помилки:\r\n           - G/Case: неправильне використання відмінка.\r\n           - G/Gender: неправильне використання роду.\r\n           - G/Number: неправильне використання числа.\r\n           - G/Aspect: неправильне використання аспекта дієслова.\r\n           - G/Tense: неправильне використання часу дієслова.\r\n           - G/VerbVoice: неправильне використання форми дієслова.\r\n           - G/PartVoice: неправильне використання форм

In [69]:
test_inference("Як варити борщ?")

'<|user|>\nЯк варити борщ?<|end|>\n<|endoftext|> Використовуючи дані з математичними формулами, розміщуйте рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні рівні

## Evaluate the performance

In [51]:
# 'datasets' is a library that provides access to a large collection of datasets and evaluation metrics for Natural Language Processing (NLP).

# 'load_metric' is a function from the 'datasets' library that loads an evaluation metric.

# 'from datasets import load_metric' imports the 'load_metric' function from the 'datasets' library. This allows you to use 'load_metric' directly instead of having to write 'datasets.load_metric'.
from datasets import load_metric

We'll employ the ROGUE metric for performance evaluation. While it may not be the optimal metric, it's straightforward and convenient to measure.

In [52]:
# 'load_metric' is a function from the 'datasets' library that loads an evaluation metric.

# 'rouge_metric = load_metric("rouge", trust_remote_code=True)' loads the ROUGE metric and assigns it to the variable 'rouge_metric'.

# The 'trust_remote_code' parameter is set to 'True', which means that the function will trust and execute remote code. This is necessary because the ROUGE metric is not included in the 'datasets' library by default and needs to be downloaded from the Hugging Face Model Hub.
rouge_metric = load_metric("rouge", trust_remote_code=True)

<ipython-input-52-cd6464720b9c>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge", trust_remote_code=True)


Establish a function for inference and evaluation of a sample.

In [53]:
# 'calculate_rogue' is a function that calculates the ROUGE score for a given row of data.

# 'row' is the input parameter, which is expected to be a dictionary-like object that contains the fields 'messages' and 'output'.

# 'response = test_inference(row['messages'][0]['content'])' generates a response by running the 'test_inference' function on the 'content' field of the first message in the 'messages' field of 'row'.

# 'result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)' computes the ROUGE score by comparing the generated 'response' with the 'output' field of 'row'. The 'use_stemmer' parameter is set to 'True', which means that the ROUGE metric will use a stemmer to reduce words to their root form before comparison.

# 'result = {key: value.mid.fmeasure * 100 for key, value in result.items()}' transforms the 'result' dictionary by multiplying each value by 100. This is done to convert the ROUGE scores from a range of 0-1 to a range of 0-100.

# 'result['response']=response' adds the generated 'response' to the 'result' dictionary.

# 'return result' returns the 'result' dictionary, which now contains the ROUGE scores and the generated response.
def calculate_rogue(row):
    response = test_inference(row['messages'][0]['content'])
    result = rouge_metric.compute(predictions=[response], references=[row['output']], use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result['response']=response
    return result

In [54]:
%%time
metricas = dataset_chatml['test'].select(range(0,500)).map(calculate_rogue, batched=False)

Parameter 'function'=<function calculate_rogue at 0x7e9faa4524d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


CPU times: user 1h 9min 13s, sys: 20 s, total: 1h 9min 33s
Wall time: 1h 9min 22s


In [55]:
# 'numpy' is a library in Python that provides support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.

# 'import numpy as np' imports the 'numpy' library and gives it the alias 'np'. This means that you can use 'np' instead of 'numpy' to refer to the library in the rest of your code.
import numpy as np

At this point, we can compute the metric for the sample.

In [56]:
# 'np.mean' is a function from the 'numpy' library that calculates the arithmetic mean of an array.

# 'metricas' is a dictionary that contains the evaluation metrics of the model.

# 'metricas['rouge1']', 'metricas['rouge2']', 'metricas['rougeL']', and 'metricas['rougeLsum']' are lists of ROUGE scores for each evaluated sample.

# ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is a set of metrics used to evaluate automatic summarization and machine translation. It compares the overlap of n-grams between the system's output and the reference summaries.

# 'print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))' prints the mean ROUGE-1 score.

# 'print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))' prints the mean ROUGE-2 score.

# 'print("Rouge L Mean: ",np.mean(metricas['rougeL']))' prints the mean ROUGE-L score.

# 'print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))' prints the mean ROUGE-Lsum score.
print("Rouge 1 Mean: ",np.mean(metricas['rouge1']))
print("Rouge 2 Mean: ",np.mean(metricas['rouge2']))
print("Rouge L Mean: ",np.mean(metricas['rougeL']))
print("Rouge Lsum Mean: ",np.mean(metricas['rougeLsum']))

Rouge 1 Mean:  0.220138397044882
Rouge 2 Mean:  0.0031342499346625357
Rouge L Mean:  0.18088117447319552
Rouge Lsum Mean:  0.17946649719644933
